In [26]:
# standard imports
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

# normal distribution
from scipy.stats import norm

# optimization and regression imports
from scipy.optimize import minimize
from scipy import stats

# data imports
import pandas_datareader as web
# IEX data import
import os
os.environ["IEX_API_KEY"] = "pk_fef828374adf49fcb91d81ee18f3fc44"
# Quandl data import
import quandl
quandl.ApiConfig.api_key = "zn5-6teANUU-aeaQrYoV"

In [27]:
import time
import matplotlib.style
from scipy.optimize import minimize

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 10)
np.set_printoptions(suppress=True)
matplotlib.style.use('classic')


In [28]:
# optimize.linprog always minimizes your target function. 
# If you want to maximize instead, you can use that max(f(x)) == -min(-f(x))

"""
1*x[1] + 2x[2] -> maximize

1*x[1] + 0*x[2] <= 5   (bounds)
0*x[1] + 1*x[2] <= 5   (bounds)
1*x[1] + 0*x[2] >= 1   (bounds)
0*x[1] + 1*x[2] >= 1   (bounds)

1*x[1] + 1*x[2] <= 6   (constraint)

"""
from scipy import optimize

result = optimize.linprog(
    c = [-1, -2], 
    A_ub=[[1, 1]], 
    b_ub=[6],
    bounds=(1, 5),
    method='simplex'
)

print(result)
print('\nresult = {}'.format(result['fun']))
print('num iterations = {}'.format(result['nit']))
print('x0 = {}, x1= {}'.format(result['x'][0] , result['x'][1] ))

     con: array([], dtype=float64)
     fun: -11.0
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([0.])
  status: 0
 success: True
       x: array([1., 5.])

result = -11.0
num iterations = 4
x0 = 1.0, x1= 5.0


In [30]:
# optimize.linprog always minimizes your target function. 
# If you want to maximize instead, you can use that max(f(x)) == -min(-f(x))

"""
13*x[1] + 23x[2] -> maximize

5*x[1] + 15*x[2] <= 480 (constraint)
4*x[1] + 4*x[2] <= 160 (constraint)
35*x[1] + 20*x[2] <= 1190 (constraint)

"""
from scipy import optimize

# linprog:
# c: coefficients to be minimized (if maximize just multiply by -1)
# ub: aX <= b
# eq: aX = b
# bounds: single tuple (min, max), Use None to indicate that there is no bound. Default: bounds are (0, None) (all decision variables are non-negative)

result = optimize.linprog(
    c = [-13, -23], 
    A_ub=[[5, 15],[4,4],[35,20]], 
    b_ub=[480,160,1190],
    bounds=None,
    method='revised simplex'
)

# output: 
# con = 1D array of results
# fun = optimal value of the objective function
print(result)


print('\nresult = {}'.format(result['fun']))
print('num iterations = {}'.format(result['nit']))
print('x0 = {}, x1= {}'.format(result['x'][0] , result['x'][1] ))

     con: array([], dtype=float64)
     fun: -800.0
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([  0.,   0., 210.])
  status: 0
 success: True
       x: array([12., 28.])

result = -800.0
num iterations = 2
x0 = 11.999999999999998, x1= 28.0


In [81]:
print('a')

a


In [5]:
url = 'https://raw.githubusercontent.com/umachkaalex/random_optimization/master/pr_data_closes.csv'

# load previous month data
all_pr_data_closes = pd.read_csv(url)
# delete columns (stocks) with zero closes
all_pr_data_closes = all_pr_data_closes.replace(0, pd.np.nan).dropna(axis=1)
# create list of Date column
all_pr_tickers = all_pr_data_closes.columns.tolist()[:-1]
# convert dataframes to numpy arrays without Date column
all_pr_data_closes = all_pr_data_closes.values[:,:-1]

pr_start_date = pd.to_datetime('11/30/2017')
pr_end_date = pd.to_datetime('12/31/2017')


In [6]:
# calculate sharpe ratio 
# (Return Portfolio minus Risk Free Rate) / Std Dev Portfolio


In [7]:
# function to return sharpe ratio, CAGR (return), and standard deviation (risk)

def calc_fin_indicators(portfolio_capital, start, end, rf_rate=0):
    # define empty dictionary 
    fin_indicators = {}
    
    # calculate the number of years needed to calculate CAGR
    pr_years = (end-start).days/365
    
    # calculate portfolio return at the end of the period divided by start of period
    gain = portfolio_capital[-1] / portfolio_capital[0]
    
    # calculate CAGR
    CAGR = (gain ** (1 / pr_years)) -1
    
    # calculate daily returns
    daily_gain = np.diff(portfolio_capital, axis=0) / portfolio_capital[:-1]
    
    # calculate standard deviation
    std = np.std(daily_gain, ddof=1)*np.sqrt(252)
    
    # calculate sharpe ratio
    sr = (CAGR - rf_rate) / std
    
    # add parameters to dictionary
    fin_indicators['sharpe'] = sr
    fin_indicators['CAGR'] = CAGR
    fin_indicators['std_dev'] = std
    
    return fin_indicators

In [8]:
def portfolio_capital_flow(close_data, st_cap, weights):
    # define the shape of closing price array
    m_shape = close_data.shape
    
    # initialize empty array to store number of shares
    num_shares_data = np.zeros(m_shape)
    
    # initialize empty array to store portfolio performance
    capital_data = np.zeros(m_shape)
    
    # loop to calculate daily portfolio positions
    for m in range(capital_data.shape[0]):
        if m==0:
            # first day of period, use initial value
            cur_cap = st_cap
            # distribute starting capital between stocks using list of weights
            capital_data[m, :] = weights*cur_cap
            # calculate number of shares held
            num_shares_data[0,:] = capital_data[m,:]/close_data[m,:]
        else:
            # not first day of period, calculate portfolio performance
            capital_data[m, :] = num_shares_data[0,:]*close_data
            
    # summarize performance
    port_performance = np.sum(capital_data, axis=1)
    return port_performance


            

In [ ]:
def algo_optimization(close_data, weights, st_cap, start, end, max_pos):
    # objective function
    def f(opt_w):
        # calculate portfolio performance
        